Below we use 7b param code llama 2 specialized for Python

Below are 3 helper functions to get the input into the model in the optimal format - arrived at through prompt engineering

In [ ]:
def find_def_until_colon(input_string):
    def_index = input_string.find('def')

    if def_index != -1:
        colon_index = input_string.find(':', def_index)

        if colon_index != -1:
            return input_string[def_index:colon_index + 1]

    return None

In [ ]:
def everything_after_def_2(input_string):
    def_index = input_string.find('def')

    if def_index != -1:
        substring_from_def = input_string[def_index:]

        return_index = substring_from_def.rfind('return')
        if return_index != -1:
            newline_index = substring_from_def.find('\n', return_index)
            if newline_index != -1:
                result = substring_from_def[:newline_index]
                return result.strip()
            else:
                result = substring_from_def[:return_index]
                return result.strip()

    return None

In [ ]:
def proper_format(prompt_in, code_in):
    # str_1 = " "
    # fun_name, str_2 = extract_args_with_function_name(code_in)
    function_index = prompt_in.find("to")
    if function_index != -1:
        result = prompt_in[function_index + len("to"):].strip()
        return result + " \n" + find_def_until_colon(code_in)
    return ""

Below we use HuggingFace

In [ ]:
pip install git+https://github.com/huggingface/transformers.git@main accelerate

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model = "codellama/CodeLlama-7b-Python-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float32,
    device_map="auto",
)

In [ ]:
def generateCode(prompt_input, code_input):
  sequences = pipeline(
      proper_format(prompt_input, code_input),
      do_sample=True,
      top_k=10,
      temperature=1,
      top_p=0.95,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      pad_token_id=tokenizer.eos_token_id,
      max_length=200,
  )
  return everything_after_def_2(sequences[0]['generated_text'])

# print(getCodeFormat("Write a python function to find the length of the shortest word.", "def len_log(list1): min=len(list1[0]) for i in list1: if len(i)<min: min=len(i) return min"))

In [ ]:
import locale
locale.getpreferredencoding = lambda: 'UTF-8'

In [ ]:
pip install jsonlines

Make the API calls and append the results to the output file

In [ ]:
import time
import jsonlines
import os
import textwrap
import signal
import time
from contextlib import contextmanager

class TimeoutException(Exception): pass

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)


import bisect
import collections
import math
import heapq
import operator
import datetime
import itertools
import cmath
import sys
import re
import array
import copy
from collections import Counter, defaultdict


def checkCorrectness(output, obj):
    try:
      exec(output)
    except:
      print("Code didnt compile")
      success = False
      return False

    assertion_code = obj['test_list']

    success = True
    for assertion in assertion_code:
        try:
            exec(assertion)
            print("Code Passed Assertions")
        except Exception as e:
            # Continue to the next example and print the error
            print(f"Error: {e}")
            print("Code Failed Assertions")
            success = False
    return success

if __name__ == '__main__':

    # #max_loop = 974
    max_loop = 300

    #run 150 next
    start_line = 150
    with jsonlines.open('mbpp_label_refined.jsonl') as reader:
        i = sum(1 for _ in reader)

    with jsonlines.open('mbpp.jsonl') as reader:
        # Loop through each example
        for count, obj in enumerate(reader):
            # Start from the last example that was run
            with jsonlines.open('mbpp_label_refined.jsonl') as reader:
                i = sum(1 for _ in reader)
            if count < start_line - 1:
                continue

            code_llama_2 = 0
            success_gpt4 = 0
            print(obj['text'])
            for _ in range(5):
                print("Running Code Llama 2")
                code_llama_response = generateCode(obj['text'], obj['code'])
                print(code_llama_response)
                if checkCorrectness(code_llama_response, obj):
                    code_llama_2 += 1

                time.sleep(5)

            obj["code_llama_response"] = code_llama_response
            obj["code_llama_success"] = code_llama_2

            with jsonlines.open('mbpp_label_refined.jsonl', mode='a') as writer:
                writer.write(obj)

            if i == max_loop:
                break


            time.sleep(5)